In [1]:
%%writefile study_one_cleaning.py

# Importing librararies
import numpy as np
import csv as csv
import pandas as pd
import math
import json
import datetime as dt
import pprint
from IPython.display import display, HTML
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.sandbox.stats.multicomp import multipletests

# Set options
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",200)
pd.set_option('display.max_colwidth', 200)


# Get datasets
df_contributors = pd.read_csv('data/contributor_5_8_18_pre_move.csv', engine='python', sep=';')
df_contributions = pd.read_csv('data//contribution_5_8_18_pre_move.csv', engine='python', sep=';')
df_slogans = pd.read_csv('data/studythingy.csv', engine='python', sep=';')

### Format Data ### 

# Rename columns and drop excess index column for contributors and contributor_id and design_id for slogans
df_contributions.columns = ['id', 'date_created', 'status', 'comparison', 'owner_id']
df_contributors.columns = ['id', 'demographics', 'index2', 'motivations']
df_slogans.columns = ['id', 'slogan', 'contributor_id', 'design_id']

df_contributors = df_contributors.drop(['index2'], axis=1)
df_slogans = df_slogans.drop(['contributor_id'], axis=1)

# Add column to slogans for study name
df_slogans['study'] = df_slogans.design_id.apply(lambda x: "Implicit Memory" if x > 12 else ("Thinking Style" if x > 6 else "Color"))

# Convert demographics and data columns for contributors and contributions, respectively, to dictionaries
df_contributors.demographics = df_contributors.demographics.apply(lambda x: json.loads(str.strip(x, '\'')))
df_contributors.motivations = df_contributors.motivations.apply(lambda x: json.loads(str.strip(x, '\'')))
df_contributions.comparison = df_contributions.comparison.apply(lambda x: json.loads(str.strip(x, '\'')))

# convert timestamps into datetime 
df_contributions['date_created'] = pd.to_datetime(df_contributions['date_created'])

### Clean Data ####

print(len(df_contributors))


# Get cheaters  
df_comments_cheated = df_contributions[df_contributions['comparison'].apply(lambda x: 'comments' in x and x['cheated'] == 'True')]


# Drop particpant 10, who clearly was me who forgot to put taken test as true
df_contributors = df_contributors.drop(10)

# Clean of anyone who did not finish the test
grouped_contributions = df_contributions.groupby('owner_id')
for contr in df_contributors.iterrows():
    choices = len(df_contributions[df_contributions['owner_id'] == contr[1].id])
    try: 
        if (not contr[1].motivations) or (contr[1][1]['taken_test'] == 'True') or (choices > 50) or (contr[1].id in df_comments_cheated['owner_id'].values):
            # Drop from contributors
            df_contributors = df_contributors[df_contributors['id'] != contr[1].id]
            # Drop all contributions from that contributor
            df_contributions = df_contributions[df_contributions['owner_id'] != contr[1].id]
    except KeyError:
        continue

# Drop all contributions from that empty contributor
df_contributors = df_contributors[df_contributors['id'] != 0]
df_contributions = df_contributions[df_contributions['owner_id'] != 0]
     

# Clean of and save comments and new slogans
df_comments = df_contributions[df_contributions['comparison'].apply(lambda x: 'comments' in x)]
df_new_slogans = df_contributions[df_contributions['comparison'].apply(lambda x: 'newSlogan' in x)]
df_contributions = df_contributions[df_contributions['comparison'].apply(lambda x: 'comments' not in x and 'newSlogan' not in x)]

# Break up motivations and demographics into seperate dataframes 
# note they are still contained in df_contributors
# but are now also seperated into full dataframes, rather than a json field
df_motivations = df_contributors['motivations'].apply(pd.Series).copy()
df_demographics = df_contributors['demographics'].apply(pd.Series).copy()

# Add in ids of participants
df_motivations['id'] = df_contributors.id
df_demographics['id'] = df_contributors.id

# Should all be the same
print("Loading, formatting, and cleaning done")
print('Contributors:', len(df_contributors), 'motivations:', len(df_motivations), 'demographics:', len(df_demographics))


# clean slogan col to just hold slogan string
df_slogans.slogan = df_slogans.slogan.apply(lambda x: json.loads(str.strip(x, '\'')))
df_slogans.slogan = df_slogans.slogan.apply(lambda x: x['slogan'])

# df_slogans = df_slogans.set_index('slogan', drop=False)


Writing study_one_cleaning.py


Writing study_one_make_df_dem_mot.py


In [15]:
# %%writefile study_one_counts.py

# dict_contributors = {k[1].id:[] for k in df_contributors.iterrows()}

# # Make new df for total count of each slogan
# df_slogan_ids = df_slogans[['design_id', 'slogan', 'study']].copy()
# df_slogan_ids['count'] = 0
# df_slogan_ids.index = df_slogan_ids['design_id']
# df_slogan_ids = df_slogan_ids.drop(['design_id'], axis=1)


# # Get the count of each slogan for each participant based on Copeland Counting 
# # At the end of this frame you should have a dict or df that contains each slogan 
# # and an associated list of the counts of how many times each participant picked that slogan

# # Reset total count
# df_slogans['total_choices'] = 0
# df_slogan_ids['count'] = 0

# # Dict to hold all slogan counts, seperated by study
# slogans = {k:[] for k in df_slogans['design_id']}
# frames = {k:[] for k in df_slogans['framing']}


# # Group contributions by a single user 
# grouped_contributions = df_contributions.groupby('owner_id')

# # Loop through each contributor:
# for contr in df_contributors.iterrows():
# #     print(contr[1].id)
#     if (contr[1].id == 37) or (contr[1].id == 39):
#         print(contr[1])
#     try:
#         # Get the choices of that contributor
#         choices = grouped_contributions.get_group(contr[1].id).comparison.apply(lambda x: x.get('choice_id'))
#         opp_choices = grouped_contributions.get_group(contr[1].id).comparison.apply(lambda x: x.get('compared_id'))

#         # Drop any NaN or none values from both lists, convert opp_choices into ints (was list types)
#         choices = [x for x in choices if str(x) != 'nan']
#         opp_choices = [x for x in opp_choices if x is not None]
#         opp_choices = [x[0] for x in opp_choices]

#         # Convert lists both into series 
#         choices = pd.Series(choices)
#         opp_choices = pd.Series(opp_choices)
        
#         # Get value counts of both (count of how many times each id appears)
#         choice_counts = choices.value_counts()
#         opp_counts = opp_choices.value_counts()
        
#         # Concat counts into the same dataframe and fill any empty spots with 0
#         df_counts = pd.concat([choice_counts, opp_counts], axis=1)
#         df_counts = df_counts.reset_index()
#         df_counts.columns = ['design_id','choices', 'opp_choices']
#         df_counts = df_counts.fillna(0)
        
#         # Get Copeland count of each slogan id, store in 'total' column
#         df_counts['total'] = df_counts['choices'] - df_counts['opp_choices']
        
#         dict_contributors[contr[1].id] = df_counts['total'].values 
        
#         # This is just number of times people voted for a slogan 
#         # does not take into account to what it was compared to
#         for i, row in df_counts.iterrows():
#             past_count = df_slogan_ids.iloc[int(row.design_id - 1)]['count']
#             df_slogan_ids.set_value(int(row.design_id), 'count', past_count + row.choices)

#         # Merge with slogan dataframe to get the actual slogan (rather than just the id)
#         df_slogan_choices = df_slogans[['id', 'slogan', 'design_id', 'framing']].merge(df_counts, on='design_id')
        
#         # add copeland count of each slogan and frame to slogan dict
#         for s in df_slogan_choices.iterrows():
#             slogans[s[1].design_id].append(s[1].total);
#             frames[s[1].framing].append(s[1].total);
        
#     # Since we took out contributors who didn't finish, have to add this catch in case of a keyerror
#     except KeyError:
#         continue

    

Overwriting study_one_counts.py


In [5]:
%%writefile study_one_combine.py

slogan_ids = df_slogans['design_id'].values
# Set up dataframe to contain the participant, relavant motivations/demographics
# This dataframe will be copied and appended multiple times - once for each slogan 
df_dem_mot = df_motivations.merge(df_demographics[['gender', 'contr_age', 'id']], on='id')

# which slogan and their rating of that slogan
df_dem_mot['slogan'] = 0
df_dem_mot['count'] = 0
df_dem_mot['framing'] = 'Help'


# Get ranks of all contributors for all slogans
df_contrib_slogan_counts = df_contributors.drop(['demographics', 'motivations'], 1)

# Make empty df for regression analysis (will be multiple copies of df_dem_mot)
X_df = pd.DataFrame(columns=df_dem_mot.columns)

# Loop through and collect each participant rank for each slogan, 
# append entire df of contributor info (plus rank given for that slogan) to X_df
for slgn in slogan_ids:
    x_copy = df_dem_mot.copy()
    x_copy['slogan'] = slgn
    x_copy['framing'] = df_slogans.loc[slgn, 'framing']
    series_counts = df_contrib_slogan_counts[['id', slgn]]
    x_copy = x_copy.merge(series_counts, on='id')
    if len(x_copy) != len(df_dem_mot):
        raise ValueError('Number of contributors is wrong!')
    x_copy['count'] = x_copy[slgn]
    x_copy = x_copy.drop(slgn, axis=1)
    X_df = X_df.append(x_copy)

    
X_df['id'] = X_df['id'].astype('int')

cols = X_df.columns.drop(['gender', 'framing']) # take gender out for a moment because you can't conver it to numeric
X_df[cols] = X_df[cols].apply(pd.to_numeric)

Overwriting study_one_combine.py
